In [1]:
from quantumnet.components import Controller, Network
from quantumnet.utils import generate_random_request, register_request
from quantumnet.objects import Request, time
import asyncio
path = "resultados/proativo/sync.csv"
rede = Network()
controlador = Controller(rede)
row, col = 3, 4
rede.set_ready_topology("Grade", row, col)
# Criação de requisições
fmin_range = (0.5, 1)
neprs_range = (2, 4)
num_requests = 10
requests = list(generate_random_request(len(rede.hosts)-1, fmin_range, neprs_range) for i in range(num_requests))

Hosts inicializados
Canais inicializados
Pares EPRs adicionados


In [2]:
hosts = rede.hosts
for alice in hosts:
    print(f"Adicionando regras para {alice}")
    for bob in hosts:
        controlador.add_match_route_rule_in_host_proactive(alice, bob, 0.5, 5)
        controlador.add_match_route_rule_in_host_proactive(alice, bob, 0.9, 5)
    rede.get_host(alice).draw_flow_table()

Adicionando regras para 0
+--------------+----------------------+--------------------+
| Match        | Route                | Rule               |
+==============+======================+====================+
| (0, 0.5, 5)  | [0]                  | BasicRuleProactive |
+--------------+----------------------+--------------------+
| (0, 0.9, 5)  | [0]                  | BasicRuleProactive |
+--------------+----------------------+--------------------+
| (1, 0.5, 5)  | [0, 1]               | BasicRuleProactive |
+--------------+----------------------+--------------------+
| (1, 0.9, 5)  | [0, 1]               | BasicRuleProactive |
+--------------+----------------------+--------------------+
| (2, 0.5, 5)  | [0, 1, 2]            | BasicRuleProactive |
+--------------+----------------------+--------------------+
| (2, 0.9, 5)  | [0, 1, 2]            | BasicRuleProactive |
+--------------+----------------------+--------------------+
| (3, 0.5, 5)  | [0, 1, 2, 3]         | BasicRuleProactive 

In [3]:
def update_rules(host):
    for match in host.flow_table.copy():
        rule = host.flow_table[match][1]
        rule.decrement_ttl()
        if rule.ttl == 0:
            if rule.hit_count == 0:
                host.flow_table.pop(match)
                print(f"Regra expirada em {host}")
            else:
                rule.hit_count = 0
                rule.ttl = 5

def update_time():
    for host in rede.hosts:
        update_rules(rede.get_host(host))

In [4]:
# Parte adicional para simulação com timeslot
def simulate_requests_with_timeslots(rede, requests, controlador, time):
    """
    Simula o processamento de requests com a introdução de timeslots.
    - Demora 1 time-slot se a regra existe.
    - Demora 3 time-slots se uma nova regra precisa ser adicionada.
    """
    for request in requests:
        print(f"[Time {time.get_current_time()}] Processando requisição {request}...")
        alice = rede.get_host(request.alice)
        rule = alice.find_rule_by_request(request)

        if rule == False:  # Caso não exista um match na tabela
            request.starttime = time
            time.increment()
            time.increment()
            time.increment()
            controlador.add_match_route_rule_in_host_reactive(request)
            rule = alice.find_rule_by_request(request)
            controlador.run_rule(rule[1])
            request.endtime = time.get_current_time()
            update_time()    
            # Registra no CSV como um novo registro
            register_request(request, "Novo Registro", path)
        else:  # Caso já exista a regra
            request.starttime = time.get_current_time()
            time.increment()  # Delay de 1 time-slot para execução
            controlador.run_rule(rule[1])
            request.endtime = time.get_current_time()
            update_time()
            # Registra no CSV como já registrado
            register_request(request, "Já Registrado", path)
        
        # Exibir informações da requisição
        print(f"Request {request}: Start Time = {request.starttime}, End Time = {request.endtime}")

simulate_requests_with_timeslots(rede, requests, controlador, time)

[Time 0] Processando requisição R-1470B0...
[BasicRuleProactive] Passo: 1
[BasicRuleProactive] Ação: CreateEPR(3-2)
[BasicRuleProactive] Ação: CreateEPR(2-6)
[BasicRuleProactive] Passo: 2
[BasicRuleProactive] Ação: Swap(3-2-6)
Running Swap(3-2-6)
Request R-1470B0: Start Time = 0, End Time = 1
[Time 1] Processando requisição R-8C2A80...
Decisão aplicada: (High Fidelity,)
Request R-8C2A80: Start Time = <quantumnet.objects.time.TimeManager object at 0x0000014B14196840>, End Time = 4
[Time 4] Processando requisição R-8620C0...
Decisão aplicada: (High Fidelity,)
Request R-8620C0: Start Time = <quantumnet.objects.time.TimeManager object at 0x0000014B14196840>, End Time = 7
[Time 7] Processando requisição R-FF73B0...
[BasicRuleProactive] Passo: 1
[BasicRuleProactive] Ação: CreateEPR(6-5)
[BasicRuleProactive] Ação: CreateEPR(5-1)
[BasicRuleProactive] Passo: 2
[BasicRuleProactive] Ação: Swap(6-5-1)
Running Swap(6-5-1)
Request R-FF73B0: Start Time = 7, End Time = 8
[Time 8] Processando requisiçã